In [26]:
from Bio import SeqIO
import re

out_src = 'datasets/current/' #specify output folder

pattern = re.compile("[^ARNDCEQGHILKMFPSTWYV]") #no B asaparctic acid or Z glutamic acid (or "X" or "O")
standard = []
non_standard= []
for idx, record in enumerate(SeqIO.parse("amp_20_pep_toolbox_raw.fasta", "fasta")):
    if pattern.search(str(record.seq)):
        non_standard.append(idx) #saving the index of the non-standard AAs for future reference
    else:
        standard.append(record.seq)
with open(out_src+'non_standard_index.txt', 'w') as f:
    for index in non_standard: f.write(str(index)+'\n')
f.close()
print("Number of sequences with only standard AA's: ",len(standard), "\t Number with non-standard AA's",len(non_standard))

Number of sequences with only standard AA's:  22129 	 Number with non-standard AA's 2000


In [27]:
len(standard) != len(set(standard)) #sets do not allow duplicate values so we check if both lists are the same size

False

In [28]:
import random
random.seed(0) # set the seed so shuffling is reproducible
#saving the standard AA only list to a new file.
file_name = 'peptide_train.txt'
shuffled_standard = random.sample(standard, len(standard))

with open(out_src+'peptide_train+test_no_shuff.txt', 'w') as f:
    for idx, line in enumerate(standard):
        f.write(str(line)+'\n')
f.close()

with open(out_src+'peptide_train.txt', 'w') as f:
    f.write('peptides'+'\n')
    for idx,line in enumerate(shuffled_standard):
        if idx <round(0.05*len(shuffled_standard)):
            continue # the first 5% will be used for testing
        f.write(str(line)+'\n')
f.close()

with open(out_src+'peptide_test.txt', 'w') as f:
    f.write('peptides'+'\n')
    for idx,line in enumerate(shuffled_standard):
        if idx <round(0.05*len(shuffled_standard)):
            f.write(str(line)+'\n')
        else:
            break
f.close()

In [31]:
import sys
sys.path.append('../../') #allows import from 2 dirs above the CWD

inputs = 'datasets/current/peptide_train.txt'
max_len = 25
save_dir = out_src
vocab_name = 'char_dict_peptide'
weights_name = 'char_weights_peptide'

%run ../../scripts/build_vocab.py --inputs $inputs --max_len $max_len --save_dir $save_dir --vocab_name $vocab_name --weights_name $weights_name

building dictionary...
calculating weights...
{'Q': 6076, 'I': 15334, 'R': 28486, 'Y': 6334, 'H': 5580, 'C': 10150, 'F': 12360, 'N': 7035, 'P': 13650, 'S': 11806, 'W': 13507, 'K': 26099, 'T': 7037, 'G': 15678, 'A': 16316, 'D': 5878, 'V': 15533, 'L': 25306, 'E': 6011, 'M': 3605, '_': 273794, '<end>': 21023}
